# Getting Data from Snowflake 

This notebook will connect to the snowflake account, and query the information_schema.tables Table.

We will create two lists
   - large_tables (Tables with  more than 400000 rows )
   - small_tables (Tables with less than 400000 rows)

Other notebooks will reference this notebook to perform the actual data loads. 

## **This Code was Built in a system that uses Apache Spark**

----------------------------------------------------------

### Install snowflake connectors

In [ ]:
!pip install snowflake-connector-python
!pip install snowflake-spark-connector

In [ ]:
from pyspark.sql import SparkSession
import snowflake.connector

### Configure connection strings and connect to Snowflake

In [ ]:
# Snowflake connection parameters
conn_params = {
    "user": "USER NAME",
    "password": "PASSWORD",
    "account": "SNOWFLAKE ACCOUNT",
    "warehouse": "WAREHOUSE NAME",
    "database": "DATABASE NAME",
    "schema": "SCHEMA NAME"
}

# Connect to Snowflake
conn = snowflake.connector.connect(**conn_params)
cursor = conn.cursor()


### Perform Queries on the Snowflake tables, and Store output into our Lists

In [ ]:
# Get list of tables in the schema
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'RAW'")
tables = [row[0] for row in cursor.fetchall()]

# Lists to store table classifications
large_tables = []
small_tables = []

# Iterate over tables to get row counts
for table in tables:
    query = f'SELECT COUNT(*) FROM "{table}"'
    cursor.execute(query)
    row_count = cursor.fetchone()[0]  # Get row count
    
    # Classify tables
    if row_count > 400000:
        large_tables.append(table)
    else:
        small_tables.append(table)


# Close connection
# cursor.close()
# conn.close()


## !!!! Cursor is left open for further use in the notebook that calls this one !!!!